# Creación de datasets

### Importando librerías

In [305]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta
from scipy.stats import truncnorm


### Función 'days_between' para calcular el la diferencia entre 2 fechas, recibe como parámetros:
- d1: fecha mínima
- d2: fecha máxima

retorna:
- un 'int' que se refiere a la diferencia en días entre las dos fechas

In [306]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days

### Función 'get_count' para calcular los valores posibles en caso de columnas numéricas, categóricas o dates, recibe como parámetros:
- col: la columna de la cual queremos calcular el total de valores

retorna:
- un 'int' el cual se refiere a la cantidad total de valores posibles

In [307]:
def get_count(col):
    if col['type'] == 'category':
        return len(col['values'])
    elif col['type'] == 'numeric':
        return col['values']['max'] - col['values']['min']
    elif col['type'] == 'date':
        return days_between(col['values']['min'], col['values']['max'])
    else:
        return None

### Función 'getRows' para calcular el número máximo de registros que tendrán el dataframe (Este función y las anteriores son implementadas o usadas solamente en los casos en los que la columna 'random' de cualquier configuración de dataframe sea False), recibe como parámetros:
- conf: la configuración del dataframe del cual queremos obtener la cantidad máxima de registros

retorna:
- un 'int' que corresponde al número mayor entre todos los posibles valores de cada columna

In [308]:
def getRows(conf):
    columns_total_values = [
        get_count(col) for col in conf['columns'] if get_count(col) is not None
    ]

    return max(columns_total_values)

### Función 'validateRandomRows' para validar que existe el elemento 'random_rows' en el caso que el elemento 'random' sea True. Recibe como parámetros:
- conf: la configuración del dataframe donde su elemento 'random' es True.

retorna:
- Lanza un excepción si el elemento no existe, caso contrario, devuelve el valor que contiene random_rows

In [309]:
def validateRandomRows(conf):
    if 'random_rows' not in conf:
        raise NameError(f"random_rows no existe en la configuracion del dataset '{conf['ds']}'")
    
    return conf['random_rows']

### Función 'generate_unique_id' para crear un str de 16 caracteres de manera aleatoria
retorna:
- un 'str' el cual es un string de 16 caracteres, que pueden ser letras y números

In [310]:
def generate_unique_id():
    caracteres = string.ascii_letters + string.digits
    unique_id = ''.join(random.choices(caracteres, k=16))
    return unique_id

### Función 'generate_unique_ids' para crear 'n' str de 16 caracteres, donde 'n' es el número de registros máximos que tendrá el dataframe dependiendo de la configuración, recibe como parámetros:
- count: el número total de 'ids' que necesitamos

retorna:
- una 'list' que contiene todos los 'ids' distintos

In [311]:
def generate_unique_ids(count):
    unique_ids = set()
    while len(unique_ids) < count:
        unique_ids.add(generate_unique_id())
    return list(unique_ids)


### Función 'generate_truncated_normal_data' para calcular valores de columnas numéricas en caso de que exista media y desviación, recibe como parámetros:
- valores estadísticos (min, max, mean y std)
- cantidad de valores que deseamos (size)

retorna:
- un objeto 'list' que contiene todos los valores calculados a partir de la información estadística proporcionada

In [312]:
def generate_truncated_normal_data(min_val, max_val, mean, std, size):
    
    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)

    # data = [int(i) for i in data]
    return data

### Función 'generate_dates' para generar una lista de fechas aleatorias. Recibe como parámetros:
- fecha_min: fecha mímima.
- fecha_max: fecha mámima.
- n: cantidad de fechas aleatorias que deseamos generar.

retorna:
- dates: lista de fechas generadas aleatoriamente dentro del rango especificado.

In [313]:
def generate_dates(fecha_min, fecha_max, n):
    # Convierte las fechas de cadena a objetos datetime
    min_date = datetime.strptime(fecha_min, '%Y-%m-%d')
    max_date = datetime.strptime(fecha_max, '%Y-%m-%d')
    
    # Calcula la diferencia en días entre las fechas de min y max
    datedif_days = max_date - min_date
    
    # Genera una lista de fechas aleatorias
    dates = []
    for i in range(n):
        # Calculamos un numero de dias aleatorios entre 0 y la cantidad de dias entre la fecha min y max
        days_aleatorios = random.randint(0, datedif_days.days)

        # Sumamos a la fecha minima, el numero de dias calculados anteriormente y lo convertimos en fecha ya que tendríamos un objeto de timo datetime.
        date = min_date + timedelta(days=days_aleatorios)
        date = date.strftime('%Y-%m-%d')
        
        dates.append(date)
    
    return dates

### Función 'generate_col' para calcular cada una de las columnas de cualquier dataframe, esta función recibe como parámetros:
- col: se refiere a la configuración de la columna que deseamos crear.
- conf_list: es la lista de las configuraciones de dataframe, nos servirá para utilizar recursividad, ya que se necesitará para los casos donde existan llave foráneas.
- rows: se refiere al número de valores totales que tendrá nuestra columna.
- df_dict: este es un diccionario de dataframes. En los casos que existan llaves foráneas, nos servirá para verificar que el dataframe al que se hace referencia, ya ha sido creado, en caso contrario, lo crearemos y lo agregaremos a 'df_dict'

retorna: 
- un objeto 'Series' de pandas, el cual es la columna que deseabamos generar

In [314]:
def generate_col(col, conf_list, rows, df_dict):
    if col['type'] == 'category':
        return random.choices(col['values'], k=rows)
    elif col['type'] == 'numeric':
        min = col['values']['min']
        max = col['values']['max']
        if 'mean' in col['values'] and 'std' in col['values']:
            mean = col['values']['mean']
            std = col['values']['std']
            return generate_truncated_normal_data(min, max, mean, std, rows)
        return [int(random.uniform(min, max)) for i in range(rows)]
    elif col['type'] == 'unique':
        return generate_unique_ids(rows)
    elif col['type'] == 'date':
        return generate_dates(col['values']['min'], col['values']['max'], rows)
    elif col['type'] == 'foreign':
        ds_name_ref, col_name_ref = col['values'].split('.')
        
        ds_names = [c['ds'] for c in conf_list]
        if ds_name_ref not in ds_names:
            raise ValueError(f"El nombre de dataset '{ds_name_ref}' no existe")
        else:
            con = [c for c in conf_list if c['ds'] == ds_name_ref]
            col_names = [col['name'] for col in con[0]['columns']]
            if col_name_ref not in col_names:
                raise ValueError(f"""
                    La columna {col_name_ref} no existe en el dataset '{ds_name_ref}'
                    Solamente puede seleccionar las columnas: {str(col_names)}
                    """)
        
        if ds_name_ref in df_dict:
            return random.choices(df_dict[ds_name_ref][col_name_ref], k=rows)
        else:
            temp_conf = [c for c in conf_list if c['ds'] == ds_name_ref]
            temp_df = build_dataframe(temp_conf[0], conf_list, df_dict)
            df_dict[ds_name_ref] = temp_df
            return random.choices(df_dict[ds_name_ref][col_name_ref], k=rows)
    else:
        raise ValueError("Tipo de columna no válida")

### Función 'build_dataframe' se utiliza para crear únicamente 1 dataframe, recibe como parámetros:
- conf: la configuración completa del dataframe a crear
- conf_list: la lista de configuraciones, servirá para enviar las configuraciones a 'generate_col' para los casos recursivos o de llave foránea.
- df_dict: enviamos la diccionarios de diccionarios para ser utilizado en 'generate_col' 

retorna:
- un objeto 'DataFrame' de pandas

In [315]:
def build_dataframe(conf, conf_list, df_dict):
    rows = validateRandomRows(conf) if conf['random'] else getRows(conf)
    df = pd.DataFrame()

    for col in conf['columns']:
        df[col['name']] = generate_col(col, conf_list, rows, df_dict)
    
    return df

### Función 'build_dataframes' sirve para crear la lista final de dataframes a partir de una lista de configuraciones, recibe solamente un parámetro:
- conf_list: lista de configuraciones de los dataframes a crear.

retorna:
- una 'list' que contiene los dataframes generados a partir de las configuraciones proporcionadas

In [316]:
def build_dataframes(conf_list):
    df_dict = {}
    result = []

    for conf in conf_list:
        df = df_dict[conf['ds']] if conf['ds'] in df_dict else build_dataframe(conf, conf_list, df_dict)
        # df_dict[conf['ds']] = df
        result.append(df)

    return result

# Ejemplo de uso

In [317]:
d1 = {
        "ds": "dataset1",
        "columns": [
            {
                "name": "segment",
                "type": "category",
                "values": ["Consumer", "Corporate", "Home Office"]
            },
            {
                "name": "id",
                "type": "unique"
            },
            {
                'name': 'test',
                'type': 'numeric',
                'values': {'min': 5, 'max':10}
            },
            {
                'name': 'date_test',
                'type': 'date',
                'values': {'min': '2024-10-01', 'max': '2024-10-15'}
            }
        ],
        "random": False
    }


d2 = {
        "ds": "dataset2",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "segment",
                "type": "foreign",
                "values": "dataset1.segment"
            },
            {
                "name": "ship_mode",
                "type": "category",
                "values": ["First Class", "Same Day", "Second Class", "Standard Class"]
            }
        ],
        "random": False
    }


d3 = {
        "ds": "dataset3",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "segment",
                "type": "foreign",
                "values": "dataset1.segment"
            },
            {
                "name": "ship_mode",
                "type": "foreign",
                "values": "dataset2.ship_mode"
            },
            {
                "name": 'country',
                'type': 'category',
                'values': ['USA', 'HON']
            },
            {
                'name': 'products_cat',
                'type': 'category',
                'values': ['Accesories', 'Art', 'Phones', 'Supplies', 'Chairs', 'Paper']
            },
            {
                'name': 'region',
                'type': 'category',
                'values': ['South', 'North']
            },
            {
                "name": "quantity",
                "type": "numeric",
                "values": {"min": 1, "max": 15}
            },
            {
                "name": "sale price",
                "type": "numeric",
                "values": {"min": 10, "max": 1500, "std": 300,
                "mean": 200}
            }
            ],
        "random": True,
        "random_rows": 1000
    }

d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "customer_id",
            "type": "unique"
        },
        {
            "name": "segment",
            "type": "foreign",
            "values": "dataset1.segment"
        },
        {
            "name": "region",
            "type": "foreign",
            "values": "dataset3.region"
        },
        {
            "name": "signup_date",
            "type": "date",
            "values": {"min": "2023-01-01", "max": "2024-01-01"}
        },
        {
            "name": "loyalty_points",
            "type": "numeric",
            "values": {"min": 0, "max": 10000}
        }
    ],
    "random": True,
    "random_rows": 800
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "order_id",
            "type": "unique"
        },
        {
            "name": "customer_id",
            "type": "foreign",
            "values": "dataset4.customer_id"
        },
        {
            "name": "product_category",
            "type": "foreign",
            "values": "dataset3.products_cat"
        },
        {
            "name": "order_date",
            "type": "date",
            "values": {"min": "2024-01-01", "max": "2024-12-31"}
        },
        {
            "name": "order_value",
            "type": "numeric",
            "values": {"min": 50, "max": 2000}
        },
        {
            "name": "shipment_mode",
            "type": "foreign",
            "values": "dataset2.ship_mode"
        }
    ],
    "random": True,
    "random_rows": 1200
}

d6 = {
    "ds": "dataset6",
    "columns": [
        {
            "name": "review_id",
            "type": "unique"
        },
        {
            "name": "order_id",
            "type": "foreign",
            "values": "dataset5.order_id"
        },
        {
            "name": "review_date",
            "type": "date",
            "values": {"min": "2024-01-01", "max": "2024-12-31"}
        },
        {
            "name": "rating",
            "type": "numeric",
            "values": {"min": 1, "max": 5}
        }
    ],
    "random": False
}



configuration_list = [d2, d3, d1, d4, d6, d5]

dataframe_list = build_dataframes(configuration_list)

dataframe_list[5]


,order_id,customer_id,product_category,order_date,order_value,shipment_mode
0,XeNZGgPGiwk5nfsX,iFlEeCAjOL4SNmZr,Art,2024-11-14,472,Same Day
1,qyOnZSZbqkLs1euE,zFH8R3lGOskywzhs,Phones,2024-12-17,1377,Same Day
2,vZB4MLo3O9fg9yLw,tv4Zv6SjYtN0ryQJ,Accesories,2024-12-16,1626,Standard Class
3,IWEU25juL2FOQFLQ,eYE2uk61fVuUeCdr,Accesories,2024-09-13,1172,Standard Class
4,7P3q4DfgW8QNq3Ju,V7FN8kKkE9Chr4m5,Supplies,2024-08-27,1936,Same Day
...,...,...,...,...,...,...
1195,5uuWMj44M8nRRgfA,RHUx2Gz8GfLiPN97,Accesories,2024-11-28,1714,Standard Class
1196,Wpya4txcSXhtHkYI,aosEwTxM8iJQKdaH,Paper,2024-04-13,1797,Standard Class
1197,rrxk9HAiC84AXaOk,tpfwgwkq763CIQXW,Accesories,2024-11-16,300,Standard Class
1198,NCxcx4KWAivW69hD,esjr9vT7qWGYUHpw,Supplies,2024-08-22,565,Standard Class


# Simulación de Dataset

In [286]:
simulation_extended = dataframe_list[1]

## Análisis Detallado

- Creamos una nueva columna para contabilizar las ocurrencias para cada combinacion distinta
- aggregation: contiene todas las distintas combinaciones entre segment', 'ship_mode', 'country' y 'products_cat', así como el total de ocurrencias que existen en el dataframe original.
- Renombramos las columnas de la agregación del paso anterior
- Calculamos las probabilidades de cada combinación, en base a la cantidad total que aparece la combinación en el dataframe original

He decidido no incluir únicamente la columna categórica 'region' ya que no me aporta información relevante para ser analizada, a diferencia de 'products_cat' que me proporciona más detalle de lo que nos habla el dataframe (que son ventas realizadas).

In [287]:


simulation_extended['n'] = 1

aggregation = (simulation_extended.
               groupby([
                   'segment',
                   'ship_mode',
                   'country',
                #    'region'
                   'products_cat'
               ], as_index = False)
               .agg({
                   'n': ['count']
               })
               )

aggregation.columns = ['segment', 'ship_mode', 'country', 'products_cat', 'count']
aggregation['probs'] = aggregation['count'] / simulation_extended.shape[0]

aggregation

,segment,ship_mode,country,products_cat,count,probs
0,Consumer,First Class,HON,Accesories,2,0.002
1,Consumer,First Class,HON,Art,6,0.006
2,Consumer,First Class,HON,Chairs,4,0.004
3,Consumer,First Class,HON,Paper,5,0.005
4,Consumer,First Class,HON,Phones,9,0.009
...,...,...,...,...,...,...
67,Home Office,Second Class,USA,Art,13,0.013
68,Home Office,Second Class,USA,Chairs,24,0.024
69,Home Office,Second Class,USA,Paper,24,0.024
70,Home Office,Second Class,USA,Phones,15,0.015


- Generamos una muestra aleatoria a partir de los indices de las combinaciones que están en 'agregation' y utilizando las probabilidades de cada combinación.
- randIndex: contiene los indices generados aleatoriamente.

In [288]:
randIndex = np.random.choice( 
    aggregation.index
    , size=10000 
    , p = list(aggregation['probs'])
)

- Visualizamos las filas del DataFrame 'agregation' cuyos índices están en randIndex, y solo incluye las columnas “segment”, "ship_mode", “country” y “products_cat”. Luego con .loc[2]: Selecciona la fila con el índice 2 de la selección anterior.

In [289]:
OneSegmentShipCountryProd = aggregation.loc[randIndex , ['segment', 'ship_mode', 'country', 'products_cat']].loc[2]

segment, ship_m, country, p_cat = OneSegmentShipCountryProd.iloc[0]

print(segment, ship_m, country, p_cat)

Consumer First Class HON Chairs


In [290]:
test = simulation_extended.loc[ 
    ( simulation_extended['segment'] == segment )
    &
    ( simulation_extended['ship_mode'] == ship_m )
    &
    ( simulation_extended['country'] == country )
    &
    ( simulation_extended['products_cat'] == p_cat )
, : ]

print(f"""
Del dataset original seleccionamos solamente los registros con las combinaciones:
- segment = {segment}
- ship_mode = {ship_m}
- country = {country}
- products_cat = {p_cat}

Y observamos que existen {test.shape[0]} registros para esa combinación. Eso significa que {test.shape[0]} / 1,000 = {test.shape[0]/1000} de representatividad en ese dataset de 1,000 registros.  
""")


Del dataset original seleccionamos solamente los registros con las combinaciones:
- segment = Consumer
- ship_mode = First Class
- country = HON
- products_cat = Chairs

Y observamos que existen 4 registros para esa combinación. Eso significa que 4 / 1,000 = 0.004 de representatividad en ese dataset de 1,000 registros.  



Ahora, al dividir los registros totales de 'OneSegmentShipCountryProd' entre 10,000, observamos que obtenemos un valor muy cercano a la representatividad del dataset original 

In [291]:
OneSegmentShipCountryProd.shape[0] / 10000


0.0047

In [292]:
salePriceStats = simulation_extended.loc[ 
    ( simulation_extended['segment'] == segment )
    &
    ( simulation_extended['ship_mode'] == ship_m )
    &
    ( simulation_extended['country'] == country )
    &
    ( simulation_extended['products_cat'] == p_cat )
, : ].groupby(
    [ "segment" , "ship_mode", 'country', 'products_cat']
    , as_index = False
).agg(
    {
        "sale price": ["min","max","mean","std"]
    }
)

salePriceStats.columns = ['segment', 'ship_mode', 'country', 'products_cat', 'min', 'max', 'mean', 'std']

salePriceStats

,segment,ship_mode,country,products_cat,min,max,mean,std
0,Consumer,First Class,HON,Chairs,250.630535,632.752664,445.153926,167.560159


In [293]:
data = generate_truncated_normal_data(
    salePriceStats["min"][0]
    , salePriceStats["max"][0]
    , salePriceStats["mean"][0]
    , salePriceStats["std"][0]
    , OneSegmentShipCountryProd.shape[0]
)

data

array([281.38794988, 563.97633827, 557.25257891, 563.88364227,
       433.00099502, 394.09569477, 359.67953693, 413.15731358,
       255.04903522, 571.59292997, 455.25566432, 338.03902452,
       523.04329901, 571.93101791, 443.8438513 , 529.49197883,
       467.22381329, 445.5721177 , 302.74502746, 498.90016856,
       357.14403195, 476.25265702, 422.31265137, 349.61529683,
       539.51406182, 499.16659916, 295.36882789, 477.87735268,
       318.14933138, 339.04515094, 597.24913683, 287.70001855,
       332.02523671, 439.59950027, 434.04873782, 367.87925657,
       551.99872591, 588.9213343 , 319.68102836, 578.88790156,
       374.14782936, 462.13230327, 312.87280578, 344.91552454,
       469.04423846, 549.3281744 , 429.63863876])

In [294]:
OneSegmentShipCountryProd['sale price'] = data

OneSegmentShipCountryProd

,segment,ship_mode,country,products_cat,sale price
2,Consumer,First Class,HON,Chairs,281.387950
2,Consumer,First Class,HON,Chairs,563.976338
2,Consumer,First Class,HON,Chairs,557.252579
2,Consumer,First Class,HON,Chairs,563.883642
2,Consumer,First Class,HON,Chairs,433.000995
2,Consumer,First Class,HON,Chairs,394.095695
2,Consumer,First Class,HON,Chairs,359.679537
2,Consumer,First Class,HON,Chairs,413.157314
2,Consumer,First Class,HON,Chairs,255.049035
2,Consumer,First Class,HON,Chairs,571.592930


## Creación del dataset simulado

In [295]:
def get_categorical_dataset_simulated(simulation_extended, category_cols, size_):
    # Encontramos las diferentes combinaciones entre las columnas categóricas seleccionadas
    simulation_extended['n'] = 1

    newAgg = simulation_extended.groupby(
        category_cols, 
        as_index = False
    ).agg(
        {
            'n': ['count']
        }
    )

    cols = [c for c in category_cols]
    cols.extend(['count'])
    newAgg.columns = cols

    # Calculamos las probabilidades de cada combinación
    newAgg['prob'] = newAgg['count'] / simulation_extended.shape[0]

    # Creamos una lista de 100,000 elementos, dichos elementos son índices calculados a partir de las diferentes combinaciones y usando sus probabilidades
    randIndex = np.random.choice(
        newAgg.index,
        size=size_,
        p= list(newAgg['prob'])
    )

    # Creamos el dataset simulado seleccionando desde newAgg, los índices que aparecen en randIndex y eligiendo unicamente las columnas categóricas búscadas
    simulated = newAgg.loc[randIndex, category_cols]

    return simulated

In [296]:
category_cols = ['segment', 'ship_mode', 'country', 'products_cat']
size = 100000

simulated = get_categorical_dataset_simulated(simulation_extended, category_cols, size)

simulated.shape

(100000, 4)

## Generando los valores de las columnas numéricas con la función utilizada en clase

In [297]:
def get_numeric_column_simulated(simulated, df_origin, categories, column_name):
    
    newAgg = df_origin.groupby(
        categories
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )
    nc = [ c for c in categories ]
    nc.extend( ["min" , "max" , "mean" , "std"] )
    newAgg.columns = nc

    ColumnSimulated = pd.DataFrame()
    for i in newAgg.index:
        rs = newAgg.loc[i]
        OneSegmentShipCountryProd = simulated.loc[i].copy()
    
        data = generate_truncated_normal_data(
            rs["min"] - 1 if rs["std"] == 0 or np.isnan(rs['std']) else rs["min"]
            , rs["max"] + 1 if rs["std"] == 0 or np.isnan(rs['std']) else rs["max"], 
            rs["mean"]
            , 1 if (rs["std"] <= 0 or np.isnan(rs['std'])) else rs["std"]
            , OneSegmentShipCountryProd.shape[0]
        )

        OneSegmentShipCountryProd[column_name] = data
    
        ColumnSimulated = pd.concat( [ ColumnSimulated , OneSegmentShipCountryProd  ] )

    return ColumnSimulated.reset_index(drop=True)



In [298]:
numeric_columns = ["quantity", 'sale price']

final_simulation = simulated.sort_index().reset_index(drop=True).copy()

for nc in numeric_columns:
    ncdf = get_numeric_column_simulated( simulated , simulation_extended , category_cols , nc )

    final_simulation = pd.merge( final_simulation , ncdf.loc[ : , [nc] ] , left_index=True , right_index=True )